In [1]:
import tensorflow as tf
import os
import argparse
from tensorflow.python.keras.callbacks import Callback


class MyFashionMnist(object):
    def train(self):

        parser = argparse.ArgumentParser()
        parser.add_argument('--learning_rate', required=False, type=float, default=0.001)
        parser.add_argument('--dropout_rate', required=False, type=float, default=0.3)
        parser.add_argument('--epoch', required=False, type=int, default=5)    
        parser.add_argument('--act', required=False, type=str, default='relu')        
        parser.add_argument('--layer', required=False, type=int, default=1)      
        parser.add_argument('--model_version', required=False, type=str, default='0001')    
        parser.add_argument('--checkpoint_dir', required=False, default='/reuslt/training_checkpoints')
        parser.add_argument('--saved_model_dir', required=False, default='/result/saved_model')        
        parser.add_argument('--tensorboard_log', required=False, default='/result/log')     
        args = parser.parse_args()    

        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
        x_train, x_test = x_train / 255.0, x_test / 255.0

        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

        for i in range(int(args.layer)):    
            model.add(tf.keras.layers.Dense(128, activation=args.act))
            if(i > 2) :
                model.add(tf.keras.layers.Dropout(args.dropout_rate))

        model.add(tf.keras.layers.Dense(10, activation='softmax'))

        model.compile(optimizer=tf.keras.optimizers.Adam(lr=args.learning_rate),
                      loss='sparse_categorical_crossentropy',
                      metrics=['acc'])

        model.summary()    

        # 체크포인트를 저장할 체크포인트 디렉터리를 지정합니다.
        checkpoint_dir = args.checkpoint_dir
        # 체크포인트 파일의 이름
        checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")        

        model.fit(x_train, y_train,
                  verbose=0,
                  validation_data=(x_test, y_test),
                  epochs=args.epoch,
                  callbacks=[KatibMetricLog(),
                            tf.keras.callbacks.TensorBoard(log_dir=args.tensorboard_log),
                            tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                   save_weights_only=True)
                            ])

        path = args.saved_model_dir + "/" + args.model_version        
        model.save(path, save_format='tf')

class KatibMetricLog(Callback):
    def on_batch_end(self, batch, logs={}):
        print("batch=" + str(batch),
              "accuracy=" + str(logs.get('acc')),
              "loss=" + str(logs.get('loss')))
    def on_epoch_begin(self, epoch, logs={}):
        print("epoch " + str(epoch) + ":")
    
    def on_epoch_end(self, epoch, logs={}):
        print("Validation-accuracy=" + str(logs.get('val_acc')),
              "Validation-loss=" + str(logs.get('val_loss')))
        return

if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils

        DOCKER_REGISTRY = 'jaewoo201'
        fairing.config.set_builder(
            'append',
            image_name='katib-job',
            base_image='jaewoo201/kubeflow-jupyter-lab:tf2.0-cpu',
            registry=DOCKER_REGISTRY, 
            push=True)
        # cpu 2, memory 5GiB
        fairing.config.set_deployer('job',
                                    namespace='kf-namespace',
                                    pod_spec_mutators=[
                                        k8s_utils.mounting_pvc(pvc_name="azurefile", 
                                                              pvc_mount_path="/result"),
                                        k8s_utils.get_resource_mutator(cpu=1,
                                                                       memory=2)]
         
                                   )
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

[W 200812 05:03:12 utils:51] The function mounting_pvc has been deprecated,                     please use `volume_mounts`
[I 200812 05:03:12 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7f10742fb3c8>
[I 200812 05:03:12 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f1026bc4160>
[I 200812 05:03:12 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f107a148470>
[W 200812 05:03:12 append:50] Building image using Append builder...
[I 200812 05:03:12 base:107] Creating docker context: /tmp/fairing_context_ls3g794z
[I 200812 05:03:12 converted_notebook:127] Converting fmnist-save-model-renew.ipynb to fmnist-save-model-renew.py
[I 200812 05:03:12 docker_creds_:234] Loading Docker credentials for repository 'jaewoo201/kubeflow-jupyter-lab:tf2.0-cpu'
[W 200812 05:03:13 append:54] Image successfully built in 0.9879837440093979s.
[W 200812 05

2020-08-12 05:03:25.677904: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-08-12 05:03:25.677949: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
4423680/4422102 [==============================] - 0s 0us/step
2020-08-12 05:03:28.758779: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2020-08-12 05:03:28.758819: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2020-08-12 05:03:28.758889: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fairing-job-gdq5b-dklw9): /proc/driver/nvidia/version does 

[W 200812 05:04:08 job:173] Cleaning up job fairing-job-gdq5b...
